In [ ]:
# DEPENDENCIES
import glob
#from shutil import copyfile
import numpy as np
import pandas as pd
#import logomaker
# LOCAL IMPORTS
#from indexing_classes import GPCRDBIndexing
import sse_func
import matplotlib.pyplot as plt
import template_finder as tf

def find_pdb(name, pdb_folder):
    identifier = name.split("-")[0]
    target_pdb = glob.glob(f"{pdb_folder}/*{identifier}*.pdb")[0]
    return target_pdb

In [ ]:
valid_collection = pd.read_pickle("../valid_collection.pkl")

In [ ]:
for i, gain in enumerate(valid_collection.collection[:10]):
    #if "A0A6J3E5X4" not in gain.name:
    #    continue
    #print(i)
    file_prefix = f"../test_stal_indexing/t_all_{i}"
    print("_"*30, f"\n{i} {gain.name}")
    #for i, hel in enumerate(gain.sda_helices):
    #    print(f"H#{i}: {hel[0]+gain.start}-{hel[1]+gain.start}")
    #for i, hel in enumerate(gain.sdb_sheets):
    #    print(f"S#{i}: {hel[0]+gain.start}-{hel[1]+gain.start}")
    element_intervals, element_centers, residue_labels, unindexed_elements, params = tf.assign_indexing(gain, 
                                                                                                file_prefix=file_prefix, 
                                                                                                gain_pdb=find_pdb(gain.name, '../all_pdbs'), 
                                                                                                template_dir='../r2_template_pdbs/', 
                                                                                                debug=False, 
                                                                                                create_pdb=False,
                                                                                                hard_cut={"S2":7,"S6":3,"H5":3},
                                                                                                patch_gps=True
                                                                                                )
    split_modes = {
        0:"No Split.",
        1:"Split by coiled residue.",
        2:"Split by disordered residue.",
        3:"Split by Proline/Glycine",
        4:"Split by hard cut.",
        5:"Overwrite by anchor priority."
    }
    if params["split_mode"] > 0:
        print(params["split_mode"], split_modes[params["split_mode"]])
    #print(element_intervals, element_centers, residue_labels, unindexed_elements, sep="\n")


In [ ]:
identifier = gain.name.split('-')[0]

out = tf.run_command(f"find ../sigmas/sigma_2/ -name \"*{identifier}*stride\"")
print("subl", out)
print("pymol", find_pdb(identifier, '../all_pdbs'))
print(gain.sequence[717-gain.start:725-gain.start])

In [ ]:
print(element_intervals, element_centers, residue_labels, unindexed_elements, split_mode, sep="\n")